# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev1 toc-item"><a href="#Subgraph-Topography" data-toc-modified-id="Subgraph-Topography-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Subgraph Topography</a></div><div class="lev2 toc-item"><a href="#Relate-Subgraph-Connection-Strength-to-Connection-Distance" data-toc-modified-id="Relate-Subgraph-Connection-Strength-to-Connection-Distance-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Relate Subgraph Connection Strength to Connection Distance</a></div><div class="lev2 toc-item"><a href="#Plot-Ranked-Topographical-Sensitivity" data-toc-modified-id="Plot-Ranked-Topographical-Sensitivity-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Plot Ranked Topographical Sensitivity</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_CoreData = '/Users/akhambhati/Remotes/CORE.fMRI_multiband.mmattar/restdata'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks'
path_SubData = path_PeriphData + '/e03b-DynFuncSubgraph-Population'
path_ExpData = path_PeriphData + '/e05b-CmpTopographySubgraph-Population'

for path in [path_CoreData, path_PeriphData, path_SubData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)     
        
if not os.path.exists('./e05b-Figures'):
    os.makedirs('./e05b-Figures')

## Generate List of Data

In [ ]:
data_dict = {'subgraph_path': '{}/NMF_Optimization.consensus_subgraph.npz'.format(path_SubData),
             'cfg_path': '{}/NMF_Optimization.CfgMatr.npz'.format(path_SubData),
             'subgraph_geom_null_path': glob.glob('{}/NMF_GeomNull.subgraph_seed-*.npz'.format(path_SubData)),
             'subgraph_edge_null_path': glob.glob('{}/NMF_EdgeNull.subgraph_seed-*.npz'.format(path_SubData))}

dist_matr = np.load('{}/geom_adj_matr.npz'.format(path_SubData))['dist_matr']  
dist_conn = convert_adj_matr_to_cfg_matr(np.expand_dims(dist_matr, axis=0)).reshape(-1)
dist_ord = np.argsort(np.argsort(dist_conn))
roi_csv = pd.read_csv('{}/Atlas/HOA112_Labels.csv'.format(path_CoreData))


# Subgraph Topography

## Relate Subgraph Connection Strength to Connection Distance

In [ ]:
# Get intra-hemispheric connections
def get_intra_hemi(matr):
    intra_Rhemi_ix, intra_Rhemi_iy = np.meshgrid(np.flatnonzero(roi_csv['Hemisphere'] == 'R'),
                                               np.flatnonzero(roi_csv['Hemisphere'] == 'R')) 
    
    intra_Lhemi_ix, intra_Lhemi_iy = np.meshgrid(np.flatnonzero(roi_csv['Hemisphere'] == 'L'),
                                               np.flatnonzero(roi_csv['Hemisphere'] == 'L')) 
    
    inter_hemi_ix, inter_hemi_iy = np.meshgrid(np.flatnonzero(roi_csv['Hemisphere'] == 'L'),
                                               np.flatnonzero(roi_csv['Hemisphere'] == 'R')) 

    conn_Rhemi = convert_adj_matr_to_cfg_matr(np.expand_dims(matr[intra_Rhemi_ix, intra_Rhemi_iy], axis=0)).reshape(-1)
    conn_Lhemi = convert_adj_matr_to_cfg_matr(np.expand_dims(matr[intra_Lhemi_ix, intra_Lhemi_iy], axis=0)).reshape(-1)
    conn_hemi = convert_adj_matr_to_cfg_matr(np.expand_dims(matr[inter_hemi_ix, inter_hemi_iy], axis=0)).reshape(-1)
    
    conn_intra = np.concatenate((conn_Rhemi, conn_Lhemi))
    conn_inter = np.concatenate((conn_hemi, conn_hemi))
    
    return conn_intra, conn_inter


%matplotlib inline

# Get Data
data_sub = np.load(data_dict['subgraph_path'], mmap_mode='r')
fac_subgraph = data_sub['fac_subnet']
fac_coef = data_sub['fac_coef']

# Get Parameters
n_fac = fac_subgraph.shape[0]
n_conn = fac_subgraph.shape[1]
n_null = len(data_dict['subgraph_geom_null_path'])
dist_intra, dist_inter = get_intra_hemi(dist_matr)

# Setup buckets
topo_sens_pop_intra = np.nan*np.zeros((n_fac))
topo_sens_pop_intra_geom_null = np.nan*np.zeros((n_fac, n_null))
topo_sens_pop_intra_edge_null = np.nan*np.zeros((n_fac, n_null))

topo_sens_pop_inter = np.nan*np.zeros((n_fac))
topo_sens_pop_inter_geom_null = np.nan*np.zeros((n_fac, n_null))
topo_sens_pop_inter_edge_null = np.nan*np.zeros((n_fac, n_null))


# Correlation between Subgraph and Distance Matrix
for f_id in xrange(n_fac):
    subg_intra, subg_inter = get_intra_hemi(convert_conn_vec_to_adj_matr(fac_subgraph[f_id, :]))
    
    rho, pval = stats.pearsonr(dist_intra, subg_intra)
    topo_sens_pop_intra[f_id] = rho
    rho, pval = stats.pearsonr(dist_inter, subg_inter)
    topo_sens_pop_inter[f_id] = rho

# Geometric Null Correlation between Subgraph and Distance Matrix
for n_i, null_path in enumerate(data_dict['subgraph_geom_null_path']):
    data_null_sub = np.load(null_path, mmap_mode='r')
    fac_subgraph = data_null_sub['fac_subnet']

    for f_id in xrange(n_fac):
        subg_intra, subg_inter = get_intra_hemi(convert_conn_vec_to_adj_matr(fac_subgraph[f_id, :]))

        rho, pval = stats.pearsonr(dist_intra, subg_intra)
        topo_sens_pop_intra_geom_null[f_id, n_i] = rho
        rho, pval = stats.pearsonr(dist_inter, subg_inter)
        topo_sens_pop_inter_geom_null[f_id, n_i] = rho

# Edge Null Correlation between Subgraph and Distance Matrix
for n_i, null_path in enumerate(data_dict['subgraph_edge_null_path']):
    data_null_sub = np.load(null_path, mmap_mode='r')
    fac_subgraph = data_null_sub['fac_subnet']

    for f_id in xrange(n_fac):
        subg_intra, subg_inter = get_intra_hemi(convert_conn_vec_to_adj_matr(fac_subgraph[f_id, :]))

        rho, pval = stats.pearsonr(dist_intra, subg_intra)
        topo_sens_pop_intra_edge_null[f_id, n_i] = rho
        rho, pval = stats.pearsonr(dist_inter, subg_inter)
        topo_sens_pop_inter_edge_null[f_id, n_i] = rho

np.savez('{}/Topographic_Sensitivity.npz'.format(path_ExpData),
         topo_sens_pop_intra=topo_sens_pop_intra,
         topo_sens_pop_intra_geom_null=topo_sens_pop_intra_geom_null,
         topo_sens_pop_intra_edge_null=topo_sens_pop_intra_edge_null,
         topo_sens_pop_inter=topo_sens_pop_inter,
         topo_sens_pop_inter_geom_null=topo_sens_pop_inter_geom_null,
         topo_sens_pop_inter_edge_null=topo_sens_pop_inter_edge_null)     

## Plot Ranked Topographical Sensitivity
$\text{Subgraph}_{TRUE}$ -- represents groups of temporally co-varying edge strengths with in-tact geometry and topology -- Topographic sensitivity of subgraphs driven by temporal structure of geometry and topology

$\text{Subgraph}_{RW}$ -- represents the ``noise floor`` of the temporal co-variance without geometry or topology -- Topographic sensitivity of subgraphs driven purely by temporal structure

$\text{Subgraph}_{GRW}$ -- represents the ``noise floor`` of the temporal co-variance with in-tact geometry, but no topology -- Topographic sensitivity of subgraphs driven purely by temporal structure of geometry, but no topology

In [ ]:
# Load Data
df = np.load('{}/Topographic_Sensitivity.npz'.format(path_ExpData))

topo_sens_pop_intra = df['topo_sens_pop_intra']
topo_sens_pop_intra_geom_null = df['topo_sens_pop_intra_geom_null']
topo_sens_pop_intra_edge_null = df['topo_sens_pop_intra_edge_null']

topo_sens_pop_inter = df['topo_sens_pop_inter']
topo_sens_pop_inter_geom_null = df['topo_sens_pop_inter_geom_null']
topo_sens_pop_inter_edge_null = df['topo_sens_pop_inter_edge_null']

n_fac, n_null = topo_sens_pop_intra_geom_null.shape

### Plot Intra-Hemisepheric Distribution
plt.figure()
ax = plt.subplot(111)

# Real Topography
ax.scatter(np.arange(n_fac)+1, np.sort(topo_sens_pop_intra),
           lw=0, color=[0.2, 0.2, 0.2])
ax.plot(np.arange(n_fac)+1, np.sort(topo_sens_pop_intra),
        color=[0.2, 0.2, 0.2])

# Geometric and Edge Null Topography
ax.fill_between(np.arange(1, n_fac+1),
                y1=np.percentile(topo_sens_pop_intra_geom_null.reshape(-1), 5),
                y2=np.percentile(topo_sens_pop_intra_geom_null.reshape(-1), 95),
                color=[1.00, 0.25, 0.25], alpha=0.2, lw=0)

ax.fill_between([1, n_fac],
                y1=np.percentile(topo_sens_pop_intra_edge_null.reshape(-1), 5),
                y2=np.percentile(topo_sens_pop_intra_edge_null.reshape(-1), 95),
                color=[0.25, 0.25, 1.00], alpha=0.2, lw=0)

# Axis Settings
ax.set_xlim([0, n_fac+1])
#ax.set_ylim([-0.5, 0.5])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Pearson Correlation\n(Edge Strength vs Euclidean Dist.)');
ax.set_xlabel('Subgraphs');
ax.set_title('Intra-Hemispheric Connections')

plt.savefig('./e05b-Figures/Topographic_Sensitivity-Intra_Hemi_Conn.svg')
plt.show()
plt.close()              


### Plot Inter-Hemisepheric Distribution
plt.figure()
ax = plt.subplot(111)

# Real Topography
ax.scatter(np.arange(n_fac)+1, np.sort(topo_sens_pop_inter),
           lw=0, color=[0.2, 0.2, 0.2])
ax.plot(np.arange(n_fac)+1, np.sort(topo_sens_pop_inter),
        color=[0.2, 0.2, 0.2])

# Geometric and Edge Null Topography
ax.fill_between([1, n_fac],
                y1=np.percentile(topo_sens_pop_inter_geom_null.reshape(-1), 5),
                y2=np.percentile(topo_sens_pop_inter_geom_null.reshape(-1), 95),
                color=[1.00, 0.25, 0.25], alpha=0.2, lw=0)

ax.fill_between([1, n_fac],
                y1=np.percentile(topo_sens_pop_inter_edge_null.reshape(-1), 5),
                y2=np.percentile(topo_sens_pop_inter_edge_null.reshape(-1), 95),
                color=[0.25, 0.25, 1.00], alpha=0.2, lw=0)

# Axis Settings
ax.set_xlim([0, n_fac+1])
#ax.set_ylim([-0.5, 0.5])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Pearson Correlation\n(Edge Strength vs Euclidean Dist.)');
ax.set_xlabel('Subgraphs');
ax.set_title('Inter-Hemispheric Connections')

plt.savefig('./e05b-Figures/Topographic_Sensitivity-Inter_Hemi_Conn.svg')
plt.show()
plt.close()    